In [ ]:
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# 1. Modify the Land use data

- convert the CRS from "epsg:4326" to "SVY21"

- fixed the invalid geometry objects

In [ ]:
# read zone, extract the zone CRS
def read_zone(file_path):
    '''
    '''
    zone_gdf = gpd.read_file(file_path, 
                             dtype = {'SUBZONE_C': str})

    zone_gdf = zone_gdf[['SUBZONE_C', 'geometry']]
    zone_gdf = zone_gdf.rename(columns = {'SUBZONE_C' : 'zone_code'})
    
    try:
        assert zone_gdf['zone_code'].unique().shape[0] == zone_gdf.shape[0]
    except:
        print('zone number:', zone_gdf['zone_code'].unique().shape[0], zone_gdf.shape[0])
        sys.exit(0)
    return zone_gdf
# =============================================================================================

In [ ]:
# read land use
def read_landuse(file_path):
    # read land use
    landuse_gdf = gpd.read_file(file_path)
    
    # drop NA columns
    landuse_gdf = landuse_gdf.dropna(axis=1, how="all")
    
    landuse_gdf = landuse_gdf[['LU_DESC', 'geometry']]
    landuse_gdf = landuse_gdf.rename(columns = {'LU_DESC' : 'type'})
    
    return landuse_gdf
# =============================================================================================

In [ ]:
# read zone
file_path = "zip://MP2019-boundray-zone-SVY21.zip!MP2019-boundray-zone-SVY21/MP2019-subzone/MP2019-subzone.shp"
zone_gdf = read_zone(file_path)
# print(zone_gdf.crs)  # "SVY21"
CRS = zone_gdf.crs

In [ ]:
# read land use
file_path = "zip://MP2019-landuse-shapefiles.zip!MP2019-landuse-shapefiles/MP2019-landuse.shp"
landuse_gdf = read_landuse(file_path)
print(landuse_gdf.crs)  # "epsg:4326"
landuse_gdf = landuse_gdf.to_crs(CRS)

In [ ]:
# invalid geometry indices of the land use
invalid_li = list(map(lambda x : not x, landuse_gdf.is_valid.to_list()))
print('number of invalid geometry before fix :', sum(invalid_li))
area_before_fix = landuse_gdf[invalid_li].area


# fix invalid geometry
landuse_gdf.loc[invalid_li, 'geometry'] = landuse_gdf.loc[invalid_li, 'geometry'].buffer(0)


# invalid geometry indices after fixed
invalid_li_ = list(map(lambda x : not x, landuse_gdf.is_valid.to_list()))
print('number of invalid geometry after fix :', sum(invalid_li_))
area_after_fix = landuse_gdf[invalid_li].area

# compute the area 
area_compare = pd.concat([area_before_fix, area_after_fix], axis=1)
area_compare['diff'] = area_after_fix - area_before_fix
area_compare

In [ ]:
# add "area" column
landuse_gdf['area'] = landuse_gdf.area

# save fixed land use
landuse_gdf.to_file('MP2019-landuse-SVY21')